# Cleaning up the data

In [1]:
# import general libraries
import pandas as pd
import numpy as np

In [2]:
# get the data
df = pd.read_csv('data/IST_corrected.csv', low_memory = False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19435 entries, 0 to 19434
Columns: 112 entries, HOSPNUM to NCB14
dtypes: float64(20), int64(29), object(63)
memory usage: 16.6+ MB


In [4]:
df.head()

,HOSPNUM,RDELAY,RCONSC,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,...,DEAD8,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14
0,1,17,D,M,69,Y,NaN,Y,Y,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,10,F,M,76,Y,NaN,Y,N,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1,43,F,F,71,N,NaN,Y,N,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,6,F,M,81,N,NaN,N,N,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,20,F,M,78,N,NaN,N,N,NaN,...,0,0,0,0,0,0,0,0,0,0


## Duplicates

There are no duplicates in the dataset.

In [5]:
# check for duplicates
double = df[df.duplicated()]
double

,HOSPNUM,RDELAY,RCONSC,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,...,DEAD8,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14


There are 112 columns in the dataset. Please see the document [IST_corrected_columns](IST_corrected_columns.md) for a detailed list of columns and which information these store. Generally, the columns are broken down into these categories: 
+ Randomization data
+ Data collected on 14 day/discharge form about treatments given in hospital
+ Final diagnisos of initial event
+ Other events within 14 days
+ Data collected at 6 months
+ Other data and derived variables

Below we will take an overview of the columns and then a more detailed look at some variables, including the generation of some new variables.

## General Overview

First we want to check which kind of data we have, how many instances per variable and how much missing data.

In [6]:
df_types = df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in df_types.items()}

{'int64': Index(['HOSPNUM', 'RDELAY', 'AGE', 'RSBP', 'HOURLOCAL', 'MINLOCAL', 'DAYLOCAL',
        'CNTRYNUM', 'DIED', 'SET14D', 'ID14', 'OCCODE', 'DEAD1', 'DEAD2',
        'DEAD3', 'DEAD4', 'DEAD5', 'DEAD6', 'DEAD7', 'DEAD8', 'H14', 'ISC14',
        'NK14', 'STRK14', 'HTI14', 'PE14', 'DVT14', 'TRAN14', 'NCB14'],
       dtype='object'),
 'float64': Index(['ONDRUG', 'DMAJNCHD', 'DSIDED', 'DRSISCD', 'DRSHD', 'DRSUNKD', 'DPED',
        'DALIVED', 'DDEADD', 'DDEADC', 'FLASTD', 'FDEADD', 'FDEADC', 'FU1_RECD',
        'FU2_DONE', 'FU1_COMP', 'TD', 'EXPDD', 'EXPD6', 'EXPD14'],
       dtype='object'),
 'object': Index(['RCONSC', 'SEX', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RHEP24',
        'RASP3', 'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7',
        'RDEF8', 'STYPE', 'RDATE', 'RXASP', 'RXHEP', 'DASP14', 'DASPLT',
        'DLH14', 'DMH14', 'DHH14', 'DSCH', 'DIVH', 'DAP', 'DOAC', 'DGORM',
        'DSTER', 'DCAA', 'DHAEMD', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DMAJNCHX',
        

In [7]:
object_col = list(df.select_dtypes(include='object').columns)
num_col = list(df.select_dtypes(include=['float64', 'int64']).columns)

### Object Columns

Here we generate a general output for each variable that is an object. In addition, we transform all categorical information in uppercase. In the variable/column description we saw, that a few variables include a category "U" (for "Unknown"). We want to check how many U-instances are there for each variable so we know if we can drop these instances entirely.

In [8]:
# Iterate over object columns
for column in object_col:
    columnSeriesObj = df[column]
    print()
    print('Colunm Name: ', column)
    print('Describe: ', columnSeriesObj.describe())
    if columnSeriesObj.nunique() <= 6:
        df[column] = df[column].str.upper()                     # all strings in uppercase
        print('Unique values: ', columnSeriesObj.unique())
        print('Value counts: ', columnSeriesObj.value_counts()) # Check Categories (particularly 'U' = 'Unknown')


Colunm Name:  RCONSC
Describe:  count     19435
unique        3
top           F
freq      14921
Name: RCONSC, dtype: object
Unique values:  ['D' 'F' 'U']
Value counts:  F    14921
D     4254
U      260
Name: RCONSC, dtype: int64

Colunm Name:  SEX
Describe:  count     19435
unique        2
top           M
freq      10407
Name: SEX, dtype: object
Unique values:  ['M' 'F']
Value counts:  M    10407
F     9028
Name: SEX, dtype: int64

Colunm Name:  RSLEEP
Describe:  count     19435
unique        2
top           N
freq      13750
Name: RSLEEP, dtype: object
Unique values:  ['Y' 'N']
Value counts:  N    13750
Y     5685
Name: RSLEEP, dtype: int64

Colunm Name:  RATRIAL
Describe:  count     18451
unique        2
top           N
freq      15282
Name: RATRIAL, dtype: object
Unique values:  [nan 'Y' 'N']
Value counts:  N    15282
Y     3169
Name: RATRIAL, dtype: int64

Colunm Name:  RCT
Describe:  count     19435
unique        2
top           Y
freq      13024
Name: RCT, dtype: object
Unique v

The 'Unknown' Category does not provide any information whatsoever. So we are planning to drop this category entirely. Though, we won't do it just now. Most values are fairly low for the unknown category. So we won't loose too much other information, but a few variables have more instances. So we will drop 'U' later in the process, when we've picked the variables we want to implement in the model.

### Numerical columns

Here we check the info for each numerical variable . Though, we have to be careful as a lot of those "numerical variables" are again cetgeorical variables. Looking at the description of variables/columns we see that moste of those have been categorized by numbers. 

In [9]:
# Iterate over numeric columns
for column in num_col:
    columnSeriesObj = df[column]
    print()
    print('Colunm Name : ', column)
    print('Describe: ', columnSeriesObj.describe())



Colunm Name :  HOSPNUM
Describe:  count    19435.000000
mean       200.055467
std        160.335093
min          1.000000
25%         60.000000
50%        159.000000
75%        319.000000
max        568.000000
Name: HOSPNUM, dtype: float64

Colunm Name :  RDELAY
Describe:  count    19435.000000
mean        20.124363
std         12.470330
min          1.000000
25%          9.000000
50%         19.000000
75%         29.000000
max         48.000000
Name: RDELAY, dtype: float64

Colunm Name :  AGE
Describe:  count    19435.000000
mean        71.715410
std         11.619714
min         16.000000
25%         65.000000
50%         73.000000
75%         80.000000
max         99.000000
Name: AGE, dtype: float64

Colunm Name :  RSBP
Describe:  count    19435.000000
mean       160.159197
std         27.610382
min         70.000000
25%        140.000000
50%        160.000000
75%        180.000000
max        295.000000
Name: RSBP, dtype: float64

Colunm Name :  HOURLOCAL
Describe:  count    19435.

## Details

Let's have a more detailed look at single variables/columns.

### Date of randomization (RDATE)

Looking at "RDATE" we assume that this information is actually stored in [Polish](https://en.wikipedia.org/wiki/Slavic_calendar).

In [10]:
df.RDATE.unique() 

array(['sty-91', 'lut-91', 'mar-91', 'kwi-91', 'maj-91', 'cze-91',
       'lip-91', 'sie-91', 'wrz-91', 'pa�-91', 'lis-91', 'gru-91',
       'sty-92', 'lut-92', 'mar-92', 'kwi-92', 'maj-92', 'cze-92',
       'lip-92', 'sie-92', 'wrz-92', 'pa�-92', 'lis-92', 'gru-92',
       'sty-93', 'lut-93', 'mar-93', 'lip-93', 'sty-94', 'wrz-93',
       'sie-93', 'gru-93', 'lis-93', 'kwi-93', 'mar-94', 'wrz-94',
       'lip-94', 'sie-94', 'maj-93', 'pa�-93', 'kwi-94', 'lut-94',
       'maj-94', 'cze-93', 'cze-94', 'gru-94', 'sty-95', 'pa�-94',
       'lis-94', 'lut-95', 'lip-95', 'maj-96', 'maj-95', 'mar-95',
       'wrz-95', 'pa�-95', 'kwi-95', 'lut-96', 'sty-96', 'cze-95',
       'sie-95', 'lis-95', 'gru-95', 'mar-96', 'kwi-96'], dtype=object)

In [11]:
#extract year from RDATE
df['RYEAR'] = df.RDATE.str.extract(r'(\d{2})', expand=False)
df.RYEAR.unique()

array(['91', '92', '93', '94', '95', '96'], dtype=object)

Month (EN) | Month (PL) | Month (in df)
:----- | ----: | -----:
January  | styczeń | sty
February  | luty  | lut
March | (brzezień) | Mar
April | kwiecień | kwi
May | (trawień) | maj
June | czerwiec | cze
July | lipiec | lip
August | sierpień | sie
September | wrzesień | wrz
Ocotber | październik | pa�
November | listopad | lis
December | grudzień | gru

We transferred the information for the months based on this table - and the assumption that the original data is in Polish. With a dictionary of these columns from Polish to the according numbers of the months.

In [12]:
# equivalents/translations for months
month_dict = {'sty': '01', 'lut': '02', 'mar': '03', 'kwi': '04', 'maj': '05', 'cze': '06',
                'lip': '07', 'sie': '08', 'wrz': '09', 'pa�': '10', 'lis': '11', 'gru': '12'}

In [13]:
# replace string in RDATE with number (according to month)
df.RDATE = df.RDATE.replace(month_dict, regex=True)

In [14]:
df.RDATE.unique()

array(['01-91', '02-91', '03-91', '04-91', '05-91', '06-91', '07-91',
       '08-91', '09-91', '10-91', '11-91', '12-91', '01-92', '02-92',
       '03-92', '04-92', '05-92', '06-92', '07-92', '08-92', '09-92',
       '10-92', '11-92', '12-92', '01-93', '02-93', '03-93', '07-93',
       '01-94', '09-93', '08-93', '12-93', '11-93', '04-93', '03-94',
       '09-94', '07-94', '08-94', '05-93', '10-93', '04-94', '02-94',
       '05-94', '06-93', '06-94', '12-94', '01-95', '10-94', '11-94',
       '02-95', '07-95', '05-96', '05-95', '03-95', '09-95', '10-95',
       '04-95', '02-96', '01-96', '06-95', '08-95', '11-95', '12-95',
       '03-96', '04-96'], dtype=object)

We probably won't need this detail for the date of randomization. So, in addition, we extracted the years of randomization only.

### Country of hospital/patient ("CNTRYNM")


In [15]:
print(df.COUNTRY.value_counts())
df.CNTRYNUM.value_counts()

UK      6257
ITAL    3437
SWIT    1631
POLA     759
NETH     728
SWED     636
AUSL     597
ARGE     545
NORW     526
SPAI     478
CZEC     466
NEW      453
PORT     415
BELG     305
TURK     286
AUST     230
INDI     229
GREE     152
SING     140
USA      131
CANA     117
HONG     115
ISRA     114
HUNG     105
SLOK      86
FINL      85
BRAS      83
SOUT      69
CHIL      65
EIRE      58
SLOV      53
DENM      35
SRI       20
ROMA      18
JAPA       9
FRAN       2
Name: COUNTRY, dtype: int64


27    6257
14    3437
25    1631
18     759
15     728
24     636
1      597
29     545
17     526
22     478
7      466
16     453
19     415
3      305
35     286
2      230
37     229
31     152
34     140
28     131
5      117
30     115
13     114
36     105
44      86
10      85
42      83
21      69
6       65
9       58
20      53
8       35
23      20
33      18
38       9
11       2
Name: CNTRYNUM, dtype: int64

The data was generated in a number of countries. However, the info is not stored in an internationally recognized format. We transformed it to the ISO standard and stored in in a new variable ("COUNTRY_ISO"):

In [16]:
df['COUNTRY_ISO'] = df['COUNTRY'].replace({'UK': 'GB', 'ITAL': 'IT', 'SWIT': 'CH', 'POLA': 'PL', 'NETH': 'NL', 
    'SWED': 'SE', 'AUSL': 'AU', 'NORW': 'NO', 'ARGE': 'AR', 'SPAI': 'ES', 'CZEC': 'CZ', 'NEW': 'NZ', 'PORT': 'PT',
    'BELG': 'BE', 'TURK': 'TR', 'AUST': 'AT', 'INDI': 'IN', 'GREE': 'GR', 'SING': 'SG', 'USA': 'USA', 'CANA': 'CA',
    'ISRA': 'IL', 'HUNG': 'HU', 'HONG': 'HK', 'SLOK': 'SK', 'FINL': 'FI', 'BRAS': 'BR', 'EIRE': 'IE', 'CHIL': 'CL',
    'SOUT': 'ZA', 'SLOV': 'SI', 'DENM': 'DK', 'ROMA': 'RO', 'SRI': 'LK', 'JAPA': 'JP', 'FRAN': 'FR'})

### Age of the patients ("AGE")


In [17]:
df.AGE.describe()

count    19435.000000
mean        71.715410
std         11.619714
min         16.000000
25%         65.000000
50%         73.000000
75%         80.000000
max         99.000000
Name: AGE, dtype: float64

The age of the patients ranges from 16 to 99. Though, the majority of the patients are between 65 and 80 years old. To improve the presentation of age-related information in graphs, we created bins for this variable (with similar size):

In [18]:
pd.qcut(x=df['AGE'], q=5, precision=0)

0        (63.0, 70.0]
1        (70.0, 76.0]
2        (70.0, 76.0]
3        (76.0, 82.0]
4        (76.0, 82.0]
             ...     
19430    (63.0, 70.0]
19431    (70.0, 76.0]
19432    (76.0, 82.0]
19433    (82.0, 99.0]
19434    (15.0, 63.0]
Name: AGE, Length: 19435, dtype: category
Categories (5, interval[float64]): [(15.0, 63.0] < (63.0, 70.0] < (70.0, 76.0] < (76.0, 82.0] < (82.0, 99.0]]

In [19]:
# new variable "AGE_bin" with information of the bin
labels = ['16 - 63', '64 - 70', '71 - 76', '77 - 82', '83 - 99']
df['AGE_bin'] = pd.qcut(x=df['AGE'], q=5, labels = labels, precision=0)

In [20]:
df.AGE_bin.value_counts()

71 - 76    4266
16 - 63    4232
77 - 82    3937
64 - 70    3570
83 - 99    3430
Name: AGE_bin, dtype: int64

### Time of randomization ("RDELAY")

Patients joined the study within 48 hours after their stroke. Again, we created similarly sized bins for different time spans (and a new variable in accordance):

In [21]:
pd.qcut(x=df['RDELAY'], q=5, precision=0)

0        (14.0, 24.0]
1         (8.0, 14.0]
2        (31.0, 48.0]
3          (0.0, 8.0]
4        (14.0, 24.0]
             ...     
19430    (31.0, 48.0]
19431    (31.0, 48.0]
19432    (14.0, 24.0]
19433    (31.0, 48.0]
19434    (14.0, 24.0]
Name: RDELAY, Length: 19435, dtype: category
Categories (5, interval[float64]): [(0.0, 8.0] < (8.0, 14.0] < (14.0, 24.0] < (24.0, 31.0] < (31.0, 48.0]]

In [22]:
# new variable "RDELAY_bin" with information of the bin
labels = ['0 - 8', '9 - 14', '15 - 24', '25 - 31', '32 - 48']
df['RDELAY_bin'] = pd.qcut(x=df['RDELAY'], q=5, labels = labels, precision=0)

In [23]:
df.RDELAY_bin.value_counts()

15 - 24    4876
0 - 8      4586
32 - 48    3835
9 - 14     3385
25 - 31    2753
Name: RDELAY_bin, dtype: int64

### Treatment during the study

There are several variables describing the heparin treatment of the study. The variable "RXHEP" stores the information to which treatment the patient was allocated to. Here we need to replace "H" with "M" as the same dose of heparin was labelled with "H" in the pre-trial and with "M" in the main trial.

In [24]:
df.RXHEP.unique()

array(['N', 'L', 'H', 'M'], dtype=object)

In [25]:
# replace H with M
df['RXHEP'].replace('H','M', inplace=True)

In [26]:
print(df.RXHEP.describe())
print(df.RXHEP.unique())

count     19435
unique        3
top           N
freq       9718
Name: RXHEP, dtype: object
['N' 'L' 'M']


Similarly, there is a variable "DHH14", which is only true for patients from the pre-trial. Those patients recevied the same Heparin dose as patients labelled with "Y" in "DMH14". We merged the variables into one:

In [27]:
df['DMH14'] = np.where(df['DMH14'].isnull() == True, df['DHH14'], df['DMH14'])

We created a new variable "RXHEP14" that stores information whether a patient received Heparin or not, regardless of the dose:

In [28]:
# new variable: RXHEP14
# Heparin given for 14 days or till death/discharge (Y, N)

conditions = [
    df['RXHEP'].eq('M') & df['DMH14'].eq('Y'),
    df['RXHEP'].eq('L') & df['DLH14'].eq('Y')]

choices = ['Y','Y']

df['RXHEP14'] = np.select(conditions, choices, default='N')

In contrast to "RXHEP14" (allocation of heparin treatement), "DMH14" and "DLH14" give information on the actual heparin treatment. Hence, we created a new variable based on these variables. The variable "HEP14" includes all patients that received heparin and in which dose these patients were administered.

In [29]:
# new variable: HEP14
# Dose Heparin given for 14 days or till death/discharge (Y, N)

conditions = [
    df['RXHEP'].eq('M') & df['DMH14'].eq('Y'),
    df['RXHEP'].eq('L') & df['DLH14'].eq('Y')]

choices = ['M','L']

df['HEP14'] = np.select(conditions, choices, default='N')

In [30]:
# Do the patients receive the treatment they were assigned to? 

print(df['RXHEP'].groupby(by = [df['HEP14']]).value_counts())
print(df['RXASP'].groupby(df['DASP14']).value_counts())

HEP14  RXHEP
L      L        4450
M      M        4242
N      N        9718
       M         614
       L         411
Name: RXHEP, dtype: int64
DASP14  RXASP
N       N        9552
        Y         775
U       N           9
        Y           5
Y       Y        8929
        N         143
Name: RXASP, dtype: int64


The Aspirin treatment was also stored in a variable that included the information whether a patients was allocated to the treatment or not ("RXASP") and a variable that gives information on the actual aspririn treatment ("DASP14"). Hence, furtheron we will use "DASP14" for aspririn treatment and "HEP14" for heparin treatment (in combination of "DLH14" and "DMH14").

We also created a new variable that included all treatments:

In [31]:
# new variable: TREAT14 (Meds given for 14 days or till death or discharge)
# Treatment: 'Aspirin','Aspirin + Medium Heparin','Aspirin + Low Heparin', 'Medium Heparin', 'Low Heparin'

conditions = [
    df['DASP14'].eq('Y') & df['HEP14'].eq('N'),     # Aspirin - no Heparin
    df['DASP14'].eq('Y') & df['HEP14'].eq('M'),     # Aspirin + Medium Heparin
    df['DASP14'].eq('Y') & df['HEP14'].eq('L'),     # Aspirin + Low Heparin
    df['HEP14'].eq('M'),                            # Medium Heparin
    df['HEP14'].eq('L')]                            # Low Heparin

choices = ['Aspirin','Aspirin + Medium Heparin','Aspirin + Low Heparin', 'Medium Heparin', 'Low Heparin']

df['TREAT14'] = np.select(conditions, choices, default='Control')

In [32]:
df.TREAT14.value_counts()

Control                     5910
Aspirin                     4833
Low Heparin                 2263
Medium Heparin              2190
Aspirin + Low Heparin       2187
Aspirin + Medium Heparin    2052
Name: TREAT14, dtype: int64

Some patients took some other medication beside the study treatment. Here, we created a variable that includes additional medication and theire according names:

In [33]:
# new variable: DMEDS
# additional medication beside ASP or HEP

conditions = [
    df['DSCH'].eq('Y'), df['DIVH'].eq('Y'), df['DAP'].eq('Y'), df['DOAC'].eq('Y'),
    df['DGORM'].eq('Y'), df['DSTER'].eq('Y'), df['DCAA'].eq('Y')]

choices = ['Subcut. Heparin','Intrav. Heparin','Antiplatelet drug','Other coanticoagulants',
    'Glycerol or mannitol','Steroids','Calcium antagonist']

df['DMEDS'] = np.select(conditions, choices, default='N')

In [34]:
df.DMEDS.value_counts()

N                         13959
Glycerol or mannitol       1774
Calcium antagonist         1467
Antiplatelet drug           616
Subcut. Heparin             473
Steroids                    454
Other coanticoagulants      365
Intrav. Heparin             327
Name: DMEDS, dtype: int64

### Types of stroke

The types of strokes the patient suffered are stored in different variables. We combined the information in new variables ("DIAG" including "no stroke", "DIAGSTR" inclduing strokes only):  

In [35]:
# new variable: DIAG
# combine all types of first stroke

conditions = [
    df['DDIAGISC'].eq('Y'), df['DDIAGHA'].eq('Y'), df['DDIAGUN'].eq('Y'), df['DNOSTRK'].eq('Y')]

choices = ['Ischaemic stroke','Haemorrhagic stroke','Indeterminate stroke','Not a stroke']

df['DIAG'] = np.select(conditions, choices, default='Unknown')

In [36]:
# new variable: DIAGSTR
# combine all types of first stroke - without DNOSTRK' 

conditions = [
    df['DDIAGISC'].eq('Y'), df['DDIAGHA'].eq('Y'), df['DDIAGUN'].eq('Y')]

choices = ['Ischaemic stroke','Haemorrhagic stroke','Indeterminate stroke']

df['DIAGSTR'] = np.select(conditions, choices, default='Unknown')

In [37]:
df['DIAG'].groupby(by = [df['DIAGSTR']]).value_counts()

DIAGSTR               DIAG                
Haemorrhagic stroke   Haemorrhagic stroke       599
Indeterminate stroke  Indeterminate stroke      992
Ischaemic stroke      Ischaemic stroke        17398
Unknown               Not a stroke              420
                      Unknown                    26
Name: DIAG, dtype: int64

Drop "Unkown" in "DIAGSTR" as these include the patients that did not suffer a stroke and a few patients with an unknown diagnosis.

In [38]:
df = df[df['DIAGSTR'] != 'Unknown']

Some patients suffered a recurrent stroke within 14 days of their first stroke. We combined the types of stroke in a new variable:

In [39]:
# new variable: RDIAG14
# combine all types of recurrent stroke (within 14 days)

conditions = [
    df['DRSISC'].eq('Y'), df['DRSH'].eq('Y'), df['DRSUNK'].eq('Y'), df['DPE'].eq('Y')]

choices = ['Ischaemic stroke','Haemorrhagic stroke','Unknown type', 'Pulmonary embolism']

df['RDIAG14'] = np.select(conditions, choices, default='None')

In [40]:
df.RDIAG14.value_counts()

None                   18107
Ischaemic stroke         412
Unknown type             256
Pulmonary embolism       120
Haemorrhagic stroke       94
Name: RDIAG14, dtype: int64

### Discharged after 14 days

Patients were either did discharged alive within 14 days or not. We created a new variable to combine the possible outcomes. First let's find out what it means if "DALIVE" equals "N". We assume that these patients were not discharged. So they are still in hospital but are not dead. To prove this we may look at the outcome after 6 months ("FDEAD" - dead after 6 months, Y/N). The patients that were annotated "N" in "DALIVE" should be included marked as not dead in ("DDEAD" - dead within 14 days) and may also be still alive after 6 months (FDEAD = N):

In [41]:
# when DALIVE = N equals "not discharged" then patients should appear in OCCODE (outcome @ 6 months; 1 - dead @ 6 months)
group_da = df.groupby(by=['DDEAD','DALIVE'])
group_da['FDEAD'].value_counts()

DDEAD  DALIVE  FDEAD
N      N       N        5546
               Y        1358
               U          20
       Y       N        9022
               Y         904
               U          33
U      U       N           1
Y      N       Y        1992
               N           1
Name: FDEAD, dtype: int64

In [42]:
# new variable: RDISCH14
# Discharged after 14 days - alive or dead?

conditions = [
    df['DALIVE'].eq('Y'), df['DDEAD'].eq('Y'), df['DALIVE'].eq('N')]

choices = ['Discharged alive from hospital', 'Dead on discharge form', 'Not discharged']

df['RDISCH14'] = np.select(conditions, choices, default= 'Unknown')

In [43]:
df.RDISCH14.value_counts()

Discharged alive from hospital    10038
Not discharged                     6943
Dead on discharge form             2002
Unknown                               6
Name: RDISCH14, dtype: int64

In [44]:
# drop "unknown"
df = df[df['RDISCH14'] != 'Unknown']

### Cause of death

As mentioned above some numerical variables are actually categorical variables. We transformed the info for cause of death according the the info for the variables/columns:

In [45]:
df['DDEADC'].replace({1: 'Initial Stroke', 2: 'Recurrent stroke (ischaemic or unknown)', 3: 'Recurrent stroke (haemorrhagic)', 
4: 'Pneumonia', 5: 'Coronary heart disease', 6: 'Pulmonary embolism', 7: 'Other vascular or unknown', 8: 'Non-vascular', 0: 'Unknown'}, inplace=True)

/Users/Nicole/Documents/Data_Science/neuefische/projects/Capstone/stroke/Stroke-Outcome/.venv/lib/python3.8/site-packages/pandas/core/series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [46]:
df['FDEADC'].replace({1: 'Initial Stroke', 2: 'Recurrent stroke (ischaemic or unknown)', 3: 'Recurrent stroke (haemorrhagic)', 
4: 'Pneumonia', 5: 'Coronary heart disease', 6: 'Pulmonary embolism', 7: 'Other vascular or unknown', 8: 'Non-vascular', 0: 'Unknown'}, inplace=True)

### Outcome after 6 months

There are three variables describing the outcome after 6 months. One is "FDENNIS" shows, whether a patient is dependent or not, "FRECOVER" shows wether a patients has recovered and "OCCODE" combines these information and includes "not recovered" and "dead". Before we compare these variables, let's transform the info on "OCCODE":

In [47]:
df['OCCODE'].replace({1: 'Dead', 2: 'Dependent', 3: 'Not recovered', 4: 'Recovered'}, inplace=True)

In [48]:
# drop '0' and '9' - not many values equal to "missing or unknown"
df.OCCODE.value_counts()

Dependent        7756
Dead             4143
Not recovered    3755
Recovered        3190
0                  90
9                  49
Name: OCCODE, dtype: int64

In [49]:
df = df[df['OCCODE'] != 0]
df = df[df['OCCODE'] != 9]

In [50]:
df.OCCODE.value_counts()

Dependent        7756
Dead             4143
Not recovered    3755
Recovered        3190
Name: OCCODE, dtype: int64

When comparing the three variables with each other, we see there are some discrepancies. Particularly, when we look at "FDENNIS". We will choose the more conservative way and use "OCCODE" for the 6 months outcome.

In [51]:
# OCCODE has more info on dependency/recovery after 6 months than FDENNIS (no unknown)
group_depen = df.groupby(by=['OCCODE', 'FDENNIS'])
print(group_depen['FRECOVER'].value_counts())

OCCODE         FDENNIS  FRECOVER
Dead           N        N              1
Dependent      U        U             15
                        N              9
               Y        N           7192
                        Y            406
                        U              8
Not recovered  N        N           3739
                        U             16
Recovered      N        Y           3184
               U        Y              6
Name: FRECOVER, dtype: int64


### Residence of patients

The place of residency was noted after 14 days and after 6 months. We decoded the information:

In [52]:
df['DPLACE'].replace({'A': 'Home', 'B': 'Relatives home', 'C': 'Residential care', 'D': 'Nursing home', 'E': 'Other hospital departments', 'U': 'Unknown'}, inplace=True)

In [53]:
df['FPLACE'].replace({'A': 'Home', 'B': 'Relatives home', 'C': 'Residential care', 'D': 'Nursing home', 'E': 'Other hospital departments', 'U': 'Unknown'}, inplace=True)

## Missing values

Before going on we will check the missing values of each variable again. We dropped some rows along the way, but we probably still have some missing values:

In [54]:
for column in df.columns:
    columnSeriesObj = df[column]
    if columnSeriesObj.isnull().sum() != 0:
        print('Column Name: ', column)
        print('Missing values : ', columnSeriesObj.isnull().sum() )

Column Name:  RATRIAL
Missing values :  971
Column Name:  RHEP24
Missing values :  340
Column Name:  RASP3
Missing values :  971
Column Name:  DASP14
Missing values :  8
Column Name:  DASPLT
Missing values :  129
Column Name:  DLH14
Missing values :  8
Column Name:  DMH14
Missing values :  8
Column Name:  DHH14
Missing values :  17873
Column Name:  ONDRUG
Missing values :  1
Column Name:  DSCH
Missing values :  286
Column Name:  DIVH
Missing values :  287
Column Name:  DAP
Missing values :  4
Column Name:  DOAC
Missing values :  4
Column Name:  DGORM
Missing values :  9
Column Name:  DSTER
Missing values :  14
Column Name:  DCAA
Missing values :  15
Column Name:  DHAEMD
Missing values :  14
Column Name:  DCAREND
Missing values :  978
Column Name:  DTHROMB
Missing values :  297
Column Name:  DMAJNCHD
Missing values :  18696
Column Name:  DMAJNCHX
Missing values :  18700
Column Name:  DSIDED
Missing values :  18224
Column Name:  DSIDEX
Missing values :  18219
Column Name:  DDIAGISC
Missi

+ Drop NAs:
    + RATRIAL not noted during trial phase (984 patients) - we will drop NAs for models if needed
    + RHEP24 and RASP3 (info on meds prior to first stroke) - drop if needed/no additional info
    + DMH14 and DLH14
    + Columns with few NAs
    
+ Drop entirely:
    + DHH14 - (was merged with DMH14)
    + FDENNIS and FRECOVER - both included in OCCODE ('dependent' after 6 months)
    + 'DMAJNCHX', 'DSIDED', 'DSIDEX', 'DNOSTRKX', 'DDEADX', 'FLASTD', 'FDEADX', 'NCCODE' - (no additional info and many missing values)

In [55]:
df = df.dropna(subset=['DASP14', 'DLH14', 'DMH14', 'ONDRUG', 'DAP', 'DOAC', 'DGORM', 'DSTER', 'DCAA', 'DHAEMD',
                        'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH', 'DALIVE', 'DDEAD', 'FDEAD'])
#df_dropped.info()

In [56]:
# drop the following columns entirely
df = df.drop(['DHH14', 'FDENNIS', 'FRECOVER', 'DMAJNCHX', 'DSIDED', 'DSIDEX', 'DNOSTRKX', 'DDEADX', 'FLASTD', 'FDEADX', 'NCCODE'], axis=1)

In [57]:
for column in df.columns:
    columnSeriesObj = df[column]
    if columnSeriesObj.isnull().sum() != 0:
        print('Column Name: ', column)
        print('Missing values : ', columnSeriesObj.isnull().sum() )

Column Name:  RATRIAL
Missing values :  958
Column Name:  RHEP24
Missing values :  338
Column Name:  RASP3
Missing values :  958
Column Name:  DASPLT
Missing values :  124
Column Name:  DSCH
Missing values :  280
Column Name:  DIVH
Missing values :  281
Column Name:  DCAREND
Missing values :  958
Column Name:  DTHROMB
Missing values :  286
Column Name:  DMAJNCHD
Missing values :  18632
Column Name:  DRSISCD
Missing values :  18371
Column Name:  DRSHD
Missing values :  18685
Column Name:  DRSUNK
Missing values :  963
Column Name:  DRSUNKD
Missing values :  18523
Column Name:  DPED
Missing values :  18658
Column Name:  DALIVED
Missing values :  8882
Column Name:  DPLACE
Missing values :  9456
Column Name:  DDEADD
Missing values :  16790
Column Name:  DDEADC
Missing values :  16495
Column Name:  FDEADD
Missing values :  14557
Column Name:  FDEADC
Missing values :  14538
Column Name:  FPLACE
Missing values :  4247
Column Name:  FAP
Missing values :  4312
Column Name:  FOAC
Missing values :

## Save the cleaned data set

In [58]:
#save clean file
df.to_csv('data/IST_corrected_clean.csv') 